In [ ]:
import pandas as pd
import numpy as np
from pycaret.regression import *

# train.csv 파일 불러오기
train_data = pd.read_csv('train.csv')

# '월'과 '일'을 결합하여 '월일' 변수 생성
train_data['월일'] = train_data['월'].astype(str) + '-' + train_data['일'].astype(str)

# '월일' 열을 datetime 형식으로 변환하며, 변환 중 오류가 발생하면 처리
def convert_to_nearest_valid_date(x):
    try:
        return pd.to_datetime(x, format='%m-%d')
    except ValueError:
        month, day = x.split('-')
        month, day = int(month), int(day)
        max_day_in_month = np.clip(day, 1, 28)  # Clip day to be within a valid range (1 to 28)
        return pd.to_datetime(f'2022-{month:02d}-{max_day_in_month:02d}')
train_data['월일'] = train_data['월일'].apply(convert_to_nearest_valid_date)

# 1월 1일로부터 며칠이 지났는지 일 수를 센 값으로 변경
train_data['일_수'] = train_data['월일'] - pd.to_datetime('01-01', format='%m-%d')
train_data['일_수'] = train_data['일_수'].dt.days

# '일 수' 값을 365로 나누어 새로운 변수 '일_수_정규화' 생성
train_data['일_수_정규화'] = train_data['일_수'] % 365

# '일 수_정규화' 변수를 sin, cos 함수를 활용하여 주기성을 나타내는 두 개의 새로운 변수 생성
train_data['월일_sin'] = np.sin(2 * np.pi * train_data['일_수_정규화'] / 365)
train_data['월일_cos'] = np.cos(2 * np.pi * train_data['일_수_정규화'] / 365)

# '월일' 열은 더 이상 필요 없으므로 삭제
train_data.drop(['월일'], axis=1, inplace=True)

# '측정 시간대' 컬럼을 시간대로 변환하여 새로운 컬럼 '시간대' 생성
def time_to_hour_category(time_str):
    if '새벽' in time_str:
        return 0  # 새벽
    elif '오전' in time_str:
        return 6  # 오전
    elif '오후' in time_str:
        return 12  # 오후
    elif '저녁' in time_str:
        return 18  # 저녁
    else:
        return None  # 그 외의 경우 (예: 결측치)

train_data['시간대'] = train_data['측정 시간대'].apply(time_to_hour_category)

# 변환에 사용되지 않을 '측정 시간대' 컬럼을 삭제
train_data.drop(['측정 시간대'], axis=1, inplace=True)

# 결측치를 평균값으로 대체
train_data = train_data.fillna(train_data.mean())

# 풍속을 예측할 특성(입력 변수)과 풍속(출력 변수)을 분리합니다.
X_train = train_data.drop(['ID', '풍속 (m/s)'], axis=1)  # 입력 변수들
y_train = train_data['풍속 (m/s)']  # 출력 변수 (풍속)

# pycaret으로 AutoML 수행
exp_reg = setup(data=train_data, target='풍속 (m/s)', session_id=42)
best_model = compare_models()

# Display model comparison results
model_results = pull()
print(model_results)

# test.csv 파일 불러오기
test_data = pd.read_csv('test.csv')

# '월'과 '일'을 결합하여 '월일' 변수 생성
test_data['월일'] = test_data['월'].astype(str) + '-' + test_data['일'].astype(str)

# '월일' 열을 datetime 형식으로 변환하며, 변환 중 오류가 발생하면 가장 가까운 날짜로 처리
test_data['월일'] = test_data['월일'].apply(convert_to_nearest_valid_date)

# 1월 1일로부터 며칠이 지났는지 일 수를 센 값으로 변경
test_data['일_수'] = test_data['월일'] - pd.to_datetime('01-01', format='%m-%d')
test_data['일_수'] = test_data['일_수'].dt.days

# '일 수' 값을 365로 나누어 새로운 변수 '일_수_정규화' 생성
test_data['일_수_정규화'] = test_data['일_수'] % 365

# '일 수_정규화' 변수를 sin, cos 함수를 활용하여 주기성을 나타내는 두 개의 새로운 변수 생성
test_data['월일_sin'] = np.sin(2 * np.pi * test_data['일_수_정규화'] / 365)
test_data['월일_cos'] = np.cos(2 * np.pi * test_data['일_수_정규화'] / 365)

# '월일' 열은 더 이상 필요 없으므로 삭제
test_data.drop(['월일'], axis=1, inplace=True)

# '측정 시간대' 컬럼을 시간대로 변환하여 새로운 컬럼 '시간대' 생성
test_data['시간대'] = test_data['측정 시간대'].apply(time_to_hour_category)

# 변환에 사용되지 않을 '측정 시간대' 컬럼을 삭제
test_data.drop(['측정 시간대'], axis=1, inplace=True)

#'시간대' 변수를 sin, cos 함수를 활용하여 주기성을 나타내는 두 개의 새로운 변수를 생성합니다.
train_data['시간대_sin'] = np.sin(2 * np.pi * train_data['시간대'] / 24)
train_data['시간대_cos'] = np.cos(2 * np.pi * train_data['시간대'] / 24)

test_data['시간대_sin'] = np.sin(2 * np.pi * test_data['시간대'] / 24)
test_data['시간대_cos'] = np.cos(2 * np.pi * test_data['시간대'] / 24)

# 결측치를 평균값으로 대체
test_data = test_data.fillna(test_data.mean())

# pycaret으로 AutoML 수행
exp_reg = setup(data=test_data, target='풍속 (m/s)', session_id=42)
y_pred = predict_model(best_model, data=test_data)  # 테스트 데이터로 풍속 예측

# Submit / 제출
submission = pd.read_csv('./sample_submission.csv')

# 풍속 예측 결과를 '풍속 (m/s)' 열에 대입합니다.
submission['풍속 (m/s)'] = y_pred['Label']

# 예측 결과를 submission.csv 양식에 맞게 저장합니다.
submission.to_csv('submission.csv', index=False)

print("풍속 예측이 완료되었습니다. 결과가 submission.csv에 저장되었습니다.")